In [ ]:
import backtrader as bt
import pandas as pd

btc_csv_parsed = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-03_UTC_with_lookback.csv",
    dtformat=('%Y-%m-%d %H:%M:%S'),
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1
)

### Testing code

In [3]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-02_UTC_with_lookback.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class EmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        # self.order= None  # To keep track of pending orders

    def next(self):
        if len(self) < self.params.long_period:
            return

        if self.crossover > 0:
            print(f"BULLISH CROSSOVER - Going LONG")
            # self.close()
            self.buy()

        elif self.crossover < 0:
            print(f"BEARISH CROSSOVER - Going SHORT")
            # self.close()
            self.sell()

    def notify_trade(self, trade):
        if trade.isclosed:
            pnl = trade.pnl  # Net profit/loss
            # pct = trade.pnlcomm / trade.price  # % after commission
            print("pnl",pnl)
            print("_________________________")
            # if pnl > 0:
            #     result = "✅ PROFIT"
            # elif pnl < 0:
            #     result = "❌ LOSS"
            # else:
            #     result = "😐 BREAKEVEN"

            # print("🔁 TRADE CLOSED")
            # print(f"Entry Price: ${trade.price:.2f}")
            # # print(f"Exit Price: ${trade.close_price:.2f}")
            # print(f"Gross PnL: ${trade.pnl:.2f}")
            # print(f"Net PnL (after commission): ${trade.pnlcomm:.2f}")
            # print(f"Result: {result}")
            print("_________________________")

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"📈 BUY EXECUTED: Price = ${order.executed.price:.2f}, Size = {order.executed.size}")
                # pass
            else:
                # print(f"📉 SELL EXECUTED: Price = ${order.executed.price:.2f}, Size = {order.executed.size}")
                pass
            print("_______________________________________________")
    

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossover)


cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission



# print(f"Starting Value: ${cerebro.broker.getvalue():.2f}")
cerebro.run()
# print(f"Final Value: ${cerebro.broker.getvalue():.2f}")

BEARISH CROSSOVER - Going SHORT
_______________________________________________
BULLISH CROSSOVER - Going LONG
📈 BUY EXECUTED: Price = $105244.20, Size = 1
_______________________________________________
pnl -257.3000000000029
_________________________
_________________________
BEARISH CROSSOVER - Going SHORT
_______________________________________________
BULLISH CROSSOVER - Going LONG
📈 BUY EXECUTED: Price = $104557.90, Size = 1
_______________________________________________
pnl -1.6999999999970896
_________________________
_________________________
BEARISH CROSSOVER - Going SHORT
_______________________________________________
BULLISH CROSSOVER - Going LONG
📈 BUY EXECUTED: Price = $104366.50, Size = 1
_______________________________________________
pnl -292.5
_________________________
_________________________


## Strategy

In [6]:
from datetime import datetime, timedelta
import numpy as np

class EMACrossoverStrategy(bt.Strategy):
    """
    EMA Crossover Strategy with dynamic parameters
    Always maintains a position - switches between long and short
    """

    params = (
        ('fast_ema_period', 9),
        ('slow_ema_period', 21),
        ('amount', 1000),
        ('leverage', 1),
        ('printlog', True),
    )

    def __init__(self):
        # Calculate EMAs
        self.fast_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.fast_ema_period
        )
        self.slow_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.slow_ema_period
        )
        
        # Crossover signals
        self.crossover = bt.indicators.CrossOver(self.fast_ema, self.slow_ema)
        
        # Track position state
        self.current_position = None  # 'long', 'short', or None
        
        # Trade tracking
        self.trade_count = 0
        self.winning_trades = 0
        self.losing_trades = 0
        self.total_pnl = 0
        
        # Store trade details
        self.trades_log = []

    def log(self, txt, dt=None):
        """Logging function"""
        if self.params.printlog:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}: {txt}')

    def next(self):
        # Skip if we don't have enough data for EMAs
        if len(self.data) < max(self.params.fast_ema_period, self.params.slow_ema_period):
            return

        current_price = self.data.close[0]

        # Check for crossover signals
        if self.crossover[0] > 0:  # Fast EMA crosses above Slow EMA (Bullish)
            if self.current_position == 'short':
                # Close short position
                self.close()
                self.log(f'CLOSE SHORT at {current_price:.2f}')

            # Open long position
            size = self.calculate_position_size()
            self.buy(size=size)
            self.current_position = 'long'
            self.log(f'OPEN LONG at {current_price:.2f}, Size: {size}')

        # Fast EMA crosses below Slow EMA (Bearish)
        elif self.crossover[0] < 0:
            if self.current_position == 'long':
                # Close long position
                self.close()
                self.log(f'CLOSE LONG at {current_price:.2f}')

            # Open short position
            size = self.calculate_position_size()
            self.sell(size=size)
            self.current_position = 'short'
            self.log(f'OPEN SHORT at {current_price:.2f}, Size: {size}')

    def calculate_position_size(self):
        """Calculate position size based on amount and leverage"""
        current_price = self.data.close[0]
        # Calculate size based on available cash and leverage
        cash = self.broker.get_cash()
        max_position_value = min(
            self.params.amount * self.params.leverage, cash)
        size = max_position_value / current_price
        return size

    def notify_trade(self, trade):
        """Called when a trade is closed"""
        if not trade.isclosed:
            return

        self.trade_count += 1
        pnl = trade.pnl
        self.total_pnl += pnl

        if pnl >= 0:
            self.winning_trades += 1
            status = 'WIN'
        else:
            self.losing_trades += 1
            status = 'LOSS'

        # Log trade details
        trade_info = {
            'trade_num': self.trade_count,
            'entry_date': bt.num2date(trade.dtopen).strftime('%Y-%m-%d %H:%M:%S'),
            'exit_date': bt.num2date(trade.dtclose).strftime('%Y-%m-%d %H:%M:%S'),
            'entry_price': trade.price,
            'exit_price': trade.pnlcomm / trade.size + trade.price if trade.size != 0 else 0,
            'size': trade.size,
            'pnl': pnl,
            'pnl_percent': (pnl / abs(trade.price * trade.size)) * 100 if trade.size != 0 else 0,
            'status': status
        }

        self.trades_log.append(trade_info)

        self.log(
            f'TRADE #{self.trade_count} - {status}: PnL: {pnl:.2f} ({trade_info["pnl_percent"]:.2f}%)')

In [ ]:
import backtrader as bt


def run_ema_backtest(csv_file, fast_ema=9, slow_ema=21, amount=1000,
                     leverage=1, initial_cash=10000):
    """
    Simple EMA crossover backtest function
    
    Parameters:
    - csv_file: path to your CSV file
    - fast_ema: fast EMA period (default 9)
    - slow_ema: slow EMA period (default 21)  
    - amount: position size
    - leverage: trading leverage
    - initial_cash: starting cash
    """

    print(f"Starting backtest with EMA({fast_ema}) x EMA({slow_ema})")

    # Create Cerebro engine
    cerebro = bt.Cerebro()

    # Add your strategy (assuming you named it EMACrossoverStrategy)
    cerebro.addstrategy(
        EMACrossoverStrategy,
        fast_ema_period=fast_ema,
        slow_ema_period=slow_ema,
        amount=amount,
        leverage=leverage
    )

    # Add your data feed (same way you're doing it)
    data = bt.feeds.GenericCSVData(
        dataname=csv_file,
        dtformat=('%Y-%m-%d %H:%M:%S'),  # Adjust format for 15min data
        datetime=0,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
        openinterest=-1
    )
    cerebro.adddata(data)

    # Set broker
    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

    # Store starting value
    start_value = cerebro.broker.getvalue()
    print(f'Starting Portfolio Value: ${start_value:,.2f}')

    # Run backtest
    results = cerebro.run()
    strategy = results[0]

    # Get final value
    final_value = cerebro.broker.getvalue()

    # Calculate results
    total_return = ((final_value - start_value) / start_value) * 100

    # Print results
    print("\n" + "="*50)
    print("BACKTEST RESULTS")
    print("="*50)
    print(f"Starting Value: ${start_value:,.2f}")
    print(f"Final Value: ${final_value:,.2f}")
    print(f"Total Return: {total_return:+.2f}%")
    print(f"Total P&L: ${final_value - start_value:+,.2f}")
    print(f"Total Trades: {strategy.trade_count}")
    print(f"Winning Trades: {strategy.winning_trades}")
    print(f"Losing Trades: {strategy.losing_trades}")

    if strategy.trade_count > 0:
        win_rate = (strategy.winning_trades / strategy.trade_count) * 100
        print(f"Win Rate: {win_rate:.1f}%")

    print("="*50)

    return {
        'start_value': start_value,
        'final_value': final_value,
        'total_return': total_return,
        'total_trades': strategy.trade_count,
        'win_rate': win_rate if strategy.trade_count > 0 else 0
    }


# Example usage:
if __name__ == "__main__":
    # Run backtest with your CSV file
    results = run_ema_backtest(
        csv_file="../data/BTCUSDT_2025-06-04_15min.csv",
        fast_ema=9,
        slow_ema=21,
        amount=1000,
        leverage=1,
        initial_cash=10000
    )

    print(f"\nFinal Return: {results['total_return']:.2f}%")

## Short Selling backtesting

In [1]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-05_UTC_with_lookback.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class EmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        self.order= None  # To keep track of pending orders

    def next(self):
        if len(self) < self.params.long_period:
            return
        
        # Skip if we have pending orders
        if self.order:
            return
        
        # Get current position
        position = self.position.size

        # Check if this is the last candle and we have an open short position
        if  len(self) == 117 and position < 0:
            print(f"🔚 LAST CANDLE - Force closing SHORT position")
            print(f"candle: {len(self)} -- price: ${self.data.close[0]:.2f}")
            self.order = self.buy(size=abs(position))
            print("____________________________________")
            return

        if self.crossover < 0:
            print(f"🔥BEARISH CROSSOVER - Going SHORT")
            print(f"candle: {len(self)} -- price: ${self.data.close[0]:.2f}  -- entry price: ${self.data.open[0]:.2f}")
             # Calculate how many units you can afford
            cash = self.broker.getcash()
            price = self.data.close[0]
            size = cash / price
            
            # print(f"Cash: ${cash:.2f}, Price: ${price:.2f}, Size: {size}")
            self.order = self.sell(size=size)
        
        elif self.crossover > 0 and position < 0:
            print(f"🔥 BULLISH CROSSOVER - Closing SHORT")
            print(
                f"candle: {len(self)} -- price: ${self.data.close[0]:.2f}  -- entry price: ${self.data.open[0]:.2f}")


            # Close the entire short position
            # print(
                # f"Current short position: {position} ---- Price: ${self.data.close[0]:.2f}")
            self.order = self.buy(size=abs(position))

            print("____________________________________")
    
    def notify_order(self, order):
        if order.status == order.Completed:
            if order.isbuy():
                print(f"BUY EXECUTED at ${order.executed.price:.2f}")
            elif order.issell():
                print(f"SELL EXECUTED at ${order.executed.price:.2f}")
        """Reset self.order when order completes"""
        if order.status in [order.Completed, order.Canceled, order.Rejected]:
            print(f"Order finished: {order.status}")
            self.order = None  # ✅ This allows new orders to be created

    def notify_trade(self, trade):

        """Called when a trade is closed"""
        if trade.isclosed:
            self.total_trades += 1
            pnl = trade.pnl
            self.total_pnl += pnl
            
            if pnl > 0:
                self.winning_trades += 1
                print(f"✅ WINNING TRADE - P&L: ${pnl:.2f}")
            else:
                print(f"❌ LOSING TRADE - P&L: ${pnl:.2f}")
            
            # # Print trade details
            # print(f"Entry Price: ${trade.price:.2f} | Exit Price: ${trade.price + trade.pnl/trade.size:.2f}")
            # print(f"Size: {trade.size} | Commission: ${trade.commission:.2f}")
            
            # # Print running statistics
            # win_rate = (self.winning_trades / self.total_trades) * 100 if self.total_trades > 0 else 0
            # print(f"📊 Total Trades: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
            # print("="*50)
        


  

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossover)


cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission


# print(f"Starting Value: ${cerebro.broker.getvalue():.2f}")
cerebro.run()
# print(f"Final Value: ${cerebro.broker.getvalue():.2f}")

🔥BEARISH CROSSOVER - Going SHORT
candle: 45 -- price: $104600.30  -- entry price: $104675.20
SELL EXECUTED at $104600.40
Order finished: 4
🔥 BULLISH CROSSOVER - Closing SHORT
candle: 61 -- price: $104854.90  -- entry price: $104764.10
____________________________________
BUY EXECUTED at $104854.90
Order finished: 4
❌ LOSING TRADE - P&L: $-24.33
🔥BEARISH CROSSOVER - Going SHORT
candle: 64 -- price: $104525.00  -- entry price: $104660.50
SELL EXECUTED at $104525.00
Order finished: 4
🔥 BULLISH CROSSOVER - Closing SHORT
candle: 69 -- price: $104772.70  -- entry price: $104755.60
____________________________________
BUY EXECUTED at $104772.70
Order finished: 4
❌ LOSING TRADE - P&L: $-23.59
🔥BEARISH CROSSOVER - Going SHORT
candle: 79 -- price: $104230.90  -- entry price: $104223.70
SELL EXECUTED at $104230.90
Order finished: 4
🔚 LAST CANDLE - Force closing SHORT position
candle: 117 -- price: $101458.60
____________________________________
BUY EXECUTED at $101458.70
Order finished: 4
✅ WINNI

### LONG POSITION

In [ ]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-05_UTC_with_lookback.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class EmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        self.order = None  # To keep track of pending orders

    def next(self):
        if len(self) < self.params.long_period:
            return

        # Skip if we have pending orders
        if self.order:
            return


        # print("__________________________")
        # print(f"Current Crossover: {self.crossover[0]} at candle {len(self)}")
        # print("__________________________")

        # Debug info (uncomment if needed)
        if self.crossover[0] != 0:
            # Get current datetime and format to show time
            current_datetime = self.data.datetime.datetime(0)
            time_str = current_datetime.strftime('%H:%M')

            print(
                f"Current Crossover: {self.crossover[0]} at candle {len(self)} - Time: {time_str}")
            # print(f"Current Price: ${current_price:.2f}")

        # Get current position
        position = self.position.size

        if len(self) == 117 and position > 0:  # Force close LONG
            print(f"🔚 LAST CANDLE - Force closing LONG position")
            self.order = self.sell(size=abs(position))

        
        if self.crossover > 0 and position == 0:
            print(f"🔥BULLISH CROSSOVER - Going LONG")
            portfolio_value = self.broker.getvalue()
            price = self.data.close[0]
            
            # Use 95% of portfolio value
            position_value = portfolio_value * 0.95
            size = int(position_value / price)
            
            if size > 0:
                self.order = self.buy(size=size)

        elif self.crossover < 0 and position > 0:  # Bearish crossover - close LONG
            print(f"🔥 BEARISH CROSSOVER - Closing LONG")
            self.order = self.sell(size=abs(position))
        

    def notify_order(self, order):
        # print(f"order status: {order.status}")
        
        if order.status == order.Completed:
            if order.isbuy():
                print(f"BUY EXECUTED at ${order.executed.price:.2f}")
            elif order.issell():
                print(f"SELL EXECUTED at ${order.executed.price:.2f}")
        elif order.status == order.Margin:
            print(f"❌ ORDER REJECTED - Insufficient Margin/Cash")
        
        # Reset order for ALL final states
        if order.status in [order.Completed, order.Canceled, order.Rejected, order.Margin]:
            print(f"Order finished: {order.status}")
            self.order = None

    def notify_trade(self, trade):
        """Called when a trade is closed"""
        if trade.isclosed:
            self.total_trades += 1
            pnl = trade.pnl
            self.total_pnl += pnl

            if pnl > 0:
                self.winning_trades += 1
                print(f"✅ WINNING TRADE - P&L: ${pnl:.2f}")
            else:
                print(f"❌ LOSING TRADE - P&L: ${pnl:.2f}")

            # # Print trade details
            # print(f"Entry Price: ${trade.price:.2f} | Exit Price: ${trade.price + trade.pnl/trade.size:.2f}")
            # print(f"Size: {trade.size} | Commission: ${trade.commission:.2f}")

            # # Print running statistics
            # win_rate = (self.winning_trades / self.total_trades) * 100 if self.total_trades > 0 else 0
            # print(f"📊 Total Trades: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
            # print("="*50)


cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossover)


cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission


print(f"--------------------------------------> Starting Value: ${cerebro.broker.getvalue():.2f}")
cerebro.run()
print(f"-------------------------------> Final Value: ${cerebro.broker.getvalue():.2f}")

--------------------------------------> Starting Value: $10000.00
🔥BULLISH CROSSOVER - Going LONG
🔥BULLISH CROSSOVER - Going LONG
🔥BULLISH CROSSOVER - Going LONG
-------------------------------> Final Value: $10000.00


## UnifiedEmaCrossover

In [24]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../data/BTCUSDT_2025-06-05_UTC_with_lookback.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class UnifiedEmaCrossover(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
        ('position_pct', 0.95),  # Use 95% of portfolio for position sizing
        ('min_cash_buffer', 100),  # Minimum cash to keep as buffer
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Trade tracking
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        # Order management
        self.order = None
        self.pending_close_order = None  # Track closing orders separately

        # Data length tracking (dynamic instead of hardcoded)
        self.data_length = None

    def start(self):
        """Called when strategy starts - get total data length"""
        # This will be set once we know the data length
        pass

    def next(self):
        # Wait for enough data for indicators
        if len(self) < self.params.long_period:
            return

        # Set data length on first valid bar (avoid hardcoding)
        if self.data_length is None:
            # We'll handle end-of-data in stop() method instead
            pass

        # Skip if we have pending orders
        if self.order is not None:
            return

        # Get current position
        position = self.position.size
        current_price = self.data.close[0]

        # Debug info (uncomment if needed)
        # print(f"Candle {len(self)}: Position={position}, Crossover={self.crossover[0]}")

        # Handle crossover signals
        if self.crossover[0] > 0:  # BULLISH CROSSOVER
            self._handle_bullish_crossover(position, current_price)

        elif self.crossover[0] < 0:  # BEARISH CROSSOVER
            self._handle_bearish_crossover(position, current_price)

    def _handle_bullish_crossover(self, position, price):
        """Handle bullish crossover - close any position and go LONG"""
        print(f"🔥 BULLISH CROSSOVER at candle {len(self)} - Price: ${price:.2f}")

        # Calculate target LONG position size
        target_long_size = self._calculate_position_size(price, 'LONG')
        
        if target_long_size <= 0:
            print(f"   ⚠️ Insufficient funds for LONG position")
            return

        # Calculate net change needed
        net_change = target_long_size - position
        
        if net_change > 0:
            print(f"   Buying {net_change} units (Current: {position} -> Target: {target_long_size})")
            self.order = self.buy(size=net_change)
        elif net_change < 0:
            print(f"   Selling {abs(net_change)} units (Current: {position} -> Target: {target_long_size})")
            self.order = self.sell(size=abs(net_change))
        else:
            print(f"   Already at target LONG position of {target_long_size} units")


    def _handle_bearish_crossover(self, position, price):
        """Handle bearish crossover - close any position and go SHORT"""
        print(f"🔥 BEARISH CROSSOVER at candle {len(self)} - Price: ${price:.2f}")

        # Calculate target SHORT position size (negative)
        target_short_size = self._calculate_position_size(price, 'SHORT')
        
        if target_short_size <= 0:
            print(f"   ⚠️ Insufficient funds for SHORT position")
            return

        # Target is negative for SHORT positions
        target_position = -target_short_size
        
        # Calculate net change needed
        net_change = target_position - position
        
        if net_change < 0:  # Need to sell (go more short)
            print(f"   Selling {abs(net_change)} units (Current: {position} -> Target: {target_position})")
            self.order = self.sell(size=abs(net_change))
        elif net_change > 0:  # Need to buy (reduce short)
            print(f"   Buying {net_change} units (Current: {position} -> Target: {target_position})")
            self.order = self.buy(size=net_change)
        else:
            print(f"   Already at target SHORT position of {target_position} units")

    def _calculate_position_size(self, price, direction):
        """Calculate position size with proper risk management"""
        try:
            portfolio_value = self.broker.getvalue()
            available_cash = self.broker.getcash()
            current_position = self.position.size
            
            # For position sizing, use portfolio value (not just available cash)
            # because we might close existing positions first
            base_value = portfolio_value
            
            # Ensure we have minimum cash buffer
            if base_value <= self.params.min_cash_buffer:
                print(f"   ⚠️ Portfolio value below minimum buffer: ${base_value:.2f}")
                return 0

            # Calculate position value using percentage of portfolio
            position_value = base_value * self.params.position_pct
            
            # For SHORT positions when we currently have LONG (or vice versa),
            # we need to account for the fact that we'll close current position first
            if direction == 'SHORT' and current_position > 0:
                # We're going LONG -> SHORT, so we'll free up cash from closing LONG
                position_value = min(position_value, portfolio_value * 0.95)
            elif direction == 'LONG' and current_position < 0:
                # We're going SHORT -> LONG, current short will be closed
                position_value = min(position_value, portfolio_value * 0.95)
            else:
                # Same direction or no position, use available cash as limit
                position_value = min(position_value, available_cash - self.params.min_cash_buffer)

            # Calculate size
            size = position_value / price  # Use int to avoid fractional shares
            
            # Validate size
            if size <= 0:
                return 0

            return size

        except Exception as e:
            print(f"   ❌ Error calculating position size: {e}")
        return 0

    def stop(self):
        """Called at the end of strategy - force close any open positions"""
        position = self.position.size

        if position != 0:
            print(f"🔚 END OF DATA - Force closing position: {position} units")
            if position > 0:
                # Close LONG position
                self.sell(size=abs(position))
            else:
                # Close SHORT position
                self.buy(size=abs(position))
            print("=" * 50)

    def notify_order(self, order):
        """Handle order notifications"""

        if order.status == order.Completed:
            action = "BUY" if order.isbuy() else "SELL"
            print(f"   ✅ {action} EXECUTED at ${order.executed.price:.2f}")

        elif order.status == order.Margin:
            print(f"   ❌ ORDER REJECTED - Insufficient Margin/Cash")

        elif order.status == order.Rejected:
            print(f"   ❌ ORDER REJECTED - {order.info}")

        elif order.status == order.Canceled:
            print(f"   ⚠️ ORDER CANCELED")

        # Reset order reference for ALL final states
        if order.status in [order.Completed, order.Canceled, order.Rejected, order.Margin]:
            self.order = None

    def notify_trade(self, trade):
        """Handle trade notifications"""
        if trade.isclosed:
            self.total_trades += 1
            pnl = trade.pnl
            self.total_pnl += pnl

            # Determine trade direction
            direction = "LONG" if trade.size > 0 else "SHORT"

            if pnl > 0:
                self.winning_trades += 1
                print(f"   ✅ WINNING {direction} TRADE - P&L: ${pnl:.2f}")
            else:
                print(f"   ❌ LOSING {direction} TRADE - P&L: ${pnl:.2f}")

            # Print running statistics
            win_rate = (self.winning_trades / self.total_trades) * 100
            print(
                f"   📊 Total: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
            print("=" * 50)


       

# Strategy execution
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(UnifiedEmaCrossover)

# Broker settings
cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission

# Enable short selling (important for crypto/forex)
cerebro.broker.set_shortcash(False)  # Don't add cash when shorting

print(f"🚀 Starting Portfolio Value: ${cerebro.broker.getvalue():.2f}")
print("=" * 50)

# Run strategy
results = cerebro.run()

print("=" * 50)
print(f"🏁 Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")


🚀 Starting Portfolio Value: $10000.00
🔥 BULLISH CROSSOVER at candle 25 - Price: $104939.00
   Buying 0.09052878338844472 units (Current: 0 -> Target: 0.09052878338844472)
   ✅ BUY EXECUTED at $104939.00
🔥 BEARISH CROSSOVER at candle 45 - Price: $104600.30
   Selling 0.18098594274662072 units (Current: 0.09052878338844472 -> Target: -0.090457159358176)
   ✅ SELL EXECUTED at $104600.40
   ❌ LOSING SHORT TRADE - P&L: $-30.65
   📊 Total: 1 | Win Rate: 0.0% | Total P&L: $-30.65
🔥 BULLISH CROSSOVER at candle 61 - Price: $104854.90
   Buying 0.1803146642019003 units (Current: -0.090457159358176 -> Target: 0.08985750484372428)
   ✅ BUY EXECUTED at $104854.90
   ❌ LOSING SHORT TRADE - P&L: $-23.02
   📊 Total: 2 | Win Rate: 0.0% | Total P&L: $-53.67
🔥 BEARISH CROSSOVER at candle 64 - Price: $104525.00
   Selling 0.1795573504409826 units (Current: 0.08985750484372429 -> Target: -0.0896998455972583)
   ✅ SELL EXECUTED at $104525.00
   ❌ LOSING SHORT TRADE - P&L: $-29.64
   📊 Total: 3 | Win Rate: 0